In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os 
import sys 

sys.path.append("../..")
import circletrack_neural as ctn
import plotting_functions as pf
import pca_ica as ica

/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedCacheFailedWarning: Failed to use cache while checking for outdated package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
## Set parameters
dpath = os.path.abspath('../../../MultiCon_AfterHours/MultiCon_EEG1/post_data/results/mc_EEG1_01/2022_10_19/minian')
fig_path = os.path.abspath('../../../MultiCon_AfterHours/MultiCon_EEG1/intermediate_figures/')
sampling_rate = 1/15

In [3]:
## Load data
spikes = ctn.open_minian(dpath)['S']
pre_spikes = spikes[:, spikes.recording_type.values == 'pre']
session_spikes = spikes[:, spikes.recording_type.values == 'session']
post_spikes = spikes[:, spikes.recording_type.values == 'post']

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_AfterHours/analysis/../../circletrack_neural.py:52: RuntimeWarning: Failed to open Zarr store with consolidated metadata, falling back to try reading non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  xr.open_zarr(pjoin(dpath, d))


In [ ]:
## Cell 0's pre recording activity
cell_num = 0
pre_time = np.arange(0, pre_spikes.shape[1] * sampling_rate, sampling_rate)
fig = pf.custom_graph_template(title='Pre Activity', x_title='Time (s)', y_title='Amplitude')
fig.add_trace(go.Scatter(x=pre_time, y=pre_spikes.values[pre_spikes.unit_id == cell_num][0], line_color='grey'))
fig.show()

In [ ]:
## Cell 0's circle track activity
session_time = np.arange(0, session_spikes.shape[1] * sampling_rate, sampling_rate)
fig = pf.custom_graph_template(title='Session Activity', x_title='Time (s)', y_title='Amplitude')
fig.add_trace(go.Scatter(x=session_time, y=session_spikes.values[session_spikes.unit_id == cell_num][0]))
fig.show()

In [ ]:
## Cell 0's post recording activity
post_time = np.arange(0, post_spikes.shape[1], sampling_rate)
fig = pf.custom_graph_template(title='Post Activity', x_title='Time (s)', y_title='Amplitude')
fig.add_trace(go.Scatter(x=post_time, y=post_spikes.values[post_spikes.unit_id == cell_num][0]))
fig.show()

In [ ]:
## Cell 0's entire activity
time = np.arange(0, spikes.shape[1] * sampling_rate, sampling_rate)
fig = pf.custom_graph_template(title='All Activity', x_title='Time (s)', y_title='Amplitude')
fig.add_trace(go.Scatter(x=time, y=spikes.values[spikes.unit_id == cell_num][0], line_color='darkgrey'))
fig.add_vline(x=time[len(pre_time)], line_width=1, line_dash='dash', line_color='turquoise', opacity=1)
fig.add_vline(x=time[len(pre_time) + len(session_time)], line_width=1, line_dash='dash', line_color='turquoise', opacity=1)
fig.show()

In [4]:
## Caclulate events
num_spikes_pre = np.count_nonzero(pre_spikes.values, axis=1)
num_spikes_session = np.count_nonzero(session_spikes.values, axis=1)
first_twenty = post_spikes.values[:, :pre_spikes.shape[1]]
num_spikes_post = np.count_nonzero(first_twenty, axis=1)
event_list = []
for i in np.arange(0, post_spikes.shape[1] / pre_spikes.shape[1] - 1, 1).astype(int):
    if i == 11:
        sliced_array = post_spikes.values[:, i*pre_spikes.shape[1]:]
        event_list.append(sliced_array)
    else:
        sliced_array = post_spikes.values[:, i*pre_spikes.shape[1]:(i+1)*pre_spikes.shape[1]]
        event_list.append(np.count_nonzero(sliced_array, axis=1))
average_events_post = np.mean(event_list, axis=0)

In [6]:
## Plot events per 20 min, first 20 minutes post activity
fig = pf.custom_graph_template(x_title='Activity (events/20min)', y_title='Count', titles=['Pre', 'Session', 'Post (first 20)'], 
                               columns=3, shared_y=True, width=1200)
## Pre
fig.add_trace(go.Histogram(x=num_spikes_pre, xbins=dict(start = 0, end = np.max(num_spikes_pre), size = 60), 
                           autobinx=False, showlegend=False, marker_color=pf.colors('pre')), row=1, col=1)
fig.add_vline(x=np.mean(num_spikes_pre), line_dash='dash', line_color='red', line_width=1.5, row=1, col=1)
## Session
fig.add_trace(go.Histogram(x=num_spikes_session, xbins=dict(start = 0, end = np.max(num_spikes_session), size = 60), 
                           autobinx=False, showlegend=False, marker_color=pf.colors('session')), row=1, col=2)
fig.add_vline(x=np.mean(num_spikes_session), line_dash='dash', line_color='red', line_width=1.5, row=1, col=2)
## Post
fig.add_trace(go.Histogram(x=num_spikes_post, xbins=dict(start = 0, end = np.max(num_spikes_post), size = 60), 
                           autobinx=False, showlegend=False, marker_color=pf.colors('post')), row=1, col=3)
fig.add_vline(x=np.mean(num_spikes_post), line_dash='dash', line_color='red', line_width=1.5, row=1, col=3)
fig.update_xaxes(matches='x')
fig.show()
fig.write_image(os.path.join(fig_path, 'pre_session_post_first20.png'))

In [7]:
## Plot events per 20 min, all post activity
fig = pf.custom_graph_template(x_title='Activity (events/20min)', y_title='Count', titles=['Pre', 'Session', 'Post'], 
                               columns=3, shared_y=True, width=1200)
## Pre
fig.add_trace(go.Histogram(x=num_spikes_pre, xbins=dict(start = 0, end = np.max(num_spikes_pre), size = 60), 
                           autobinx=False, showlegend=False, marker_color=pf.colors('pre')), row=1, col=1)
fig.add_vline(x=np.mean(num_spikes_pre), line_dash='dash', line_color='red', line_width=1.5, row=1, col=1)
## Session
fig.add_trace(go.Histogram(x=num_spikes_session, xbins=dict(start = 0, end = np.max(num_spikes_session), size = 60), 
                           autobinx=False, showlegend=False, marker_color=pf.colors('session')), row=1, col=2)
fig.add_vline(x=np.mean(num_spikes_session), line_dash='dash', line_color='red', line_width=1.5, row=1, col=2)
## Post
fig.add_trace(go.Histogram(x=average_events_post, xbins=dict(start = 0, end = np.max(average_events_post), size = 60), 
                           autobinx=False, showlegend=False, marker_color=pf.colors('post')), row=1, col=3)
fig.add_vline(x=np.mean(average_events_post), line_dash='dash', line_color='red', line_width=1.5, row=1, col=3)
fig.update_xaxes(matches='x')
fig.show()
fig.write_image(os.path.join(fig_path, 'pre_session_post_all20.png'))